## Análise do Consumo Energético e Clustering de Consumidores

### Introdução

Neste notebook, serão aplicadas técnicas de modelagem de dados para analisar e segmentar padrões de consumo de energia. O objetivo principal é identificar diferentes perfis de consumo e entender como os clientes utilizam a energia ao longo do tempo. Para isso, foram utilizadas técnicas de agrupamento (clustering) para segmentar os clientes em grupos com comportamentos semelhantes de consumo de energia.

##### Inicialização da Sessão Spark

In [14]:
#import findspark
#findspark.init()
#findspark.find()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml import PipelineModel
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeansModel

# Initialize Spark session
spark = SparkSession\
        .builder\
        .appName("TrabalhoFinal")\
        .config("spark.sql.shuffle.partitions",6)\
        .config("spark.sql.repl.eagereval.enabled",True)\
        .getOrCreate()




#### Funções Auxiliares
Esta função `compute_energy_stats` calcula várias estatísticas de consumo de energia. 


1. **Estatísticas de consumo geral**: Calcula a média, máximo e mínimo do consumo de energia para cada ID de cliente (LCLid).

2. **Consumo médio para cada período do dia**: Calcula a média do consumo de energia para cada período do dia (manhã, tarde, noite, etc.) para cada cliente.

3. **Estatísticas de consumo mínimo e máximo por dia**: Calcula a soma do consumo de energia para cada dde e, em seguida, calcula o consumo mínimo e máximo para cada cliente.

4. **Combinação de todas as estatnicas em um DataFrame final**: Une todas as estatísticas calnadas em um único DataFrame final usando o ID do cliente como chave des futuras.

In [15]:
def compute_energy_stats(df):
    """
    Function to compute various energy statistics.

    Parameters:
    df (DataFrame): The input Spark DataFrame.

    Returns:
    DataFrame: The resulting DataFrame with aggregated statistics.
    """
    # Calculate overall consumption statistics
    overall_stats = df.groupBy("LCLid").agg(
        F.mean("energy(kWh/hh)").cast("float").alias("Consume_mean"),
        F.max("energy(kWh/hh)").cast("float").alias("Consume_max"),
        F.min("energy(kWh/hh)").cast("float").alias("Consume_min")
    )

    # Calculate mean consumption for each period of the day
    period_means = df.groupBy("LCLid").pivot("period_of_day").agg(
        F.mean("energy(kWh/hh)").cast("float").alias("Consume_mean")
    ).select(
        F.col("LCLid"),
        F.col("Morning").alias("Consume_mean_Morning"),
        F.col("Noon").alias("Consume_mean_Noon"),
        F.col("Evening").alias("Consume_mean_Evening"),
        F.col("Night").alias("Consume_mean_Night")
    )

    # Calculate minimum and maximum consumption day
    day_stats = df.groupBy("LCLid", "day").agg(
        F.sum("energy(kWh/hh)").cast("float").alias("daily_energy")
    ).groupBy("LCLid").agg(
        F.min("daily_energy").cast("float").alias("Min_Day"),
        F.max("daily_energy").cast("float").alias("Max_Day")
    )

    # Combine all statistics into one DataFrame
    final_df = overall_stats.join(period_means, on="LCLid", how="left") \
                            .join(day_stats, on="LCLid", how="left")

    return final_df

#### Leitura e Preparação dos Dados
Carregaram-se os dados de consumo energético e meteorológicos e realizaram-se as devidas conversões de tipo.

In [16]:
dados_final = spark.read.parquet('dados_final_parquet')
dados_final = dados_final.withColumn('temperature', F.col('temperature').cast('float'))
dados_final = dados_final.withColumn('apparentTemperature', F.col('apparentTemperature').cast('float'))
dados_final = dados_final.withColumn('humidity', F.col('humidity').cast('float'))


In [17]:
final_data = dados_final.sample(False, 0.1, seed=4123)

final_data.count()

9303749

A amostra dos dados contém 9303749 registos, representando cerca de 10% do total

Estatísticas de consumo energético utilizando a função `compute_energy_stats`.

In [18]:
final_data_all = compute_energy_stats(final_data)
final_data_all.show(5, truncate=False,vertical=True)

-RECORD 0---------------------------
 LCLid                | MAC000002   
 Consume_mean         | 0.24523506  
 Consume_max          | 2.994       
 Consume_min          | 0.065       
 Consume_mean_Morning | 0.19983937  
 Consume_mean_Noon    | 0.22717202  
 Consume_mean_Evening | 0.38890687  
 Consume_mean_Night   | 0.15912384  
 Min_Day              | 5.896       
 Max_Day              | 19.128      
-RECORD 1---------------------------
 LCLid                | MAC000003   
 Consume_mean         | 0.39423126  
 Consume_max          | 3.443       
 Consume_min          | 0.007       
 Consume_mean_Morning | 0.34106544  
 Consume_mean_Noon    | 0.16044568  
 Consume_mean_Evening | 0.15154716  
 Consume_mean_Night   | 0.9792494   
 Min_Day              | 11.1449995  
 Max_Day              | 35.382      
-RECORD 2---------------------------
 LCLid                | MAC000004   
 Consume_mean         | 0.03528794  
 Consume_max          | 0.672       
 Consume_min          | 0.0         
 

Estatísticas de consumo de energia:

**Consume_mean:** Esta coluna representa a média do consumo de energia para cada cliente.

**Consume_max:** Indica o máximo de consumo de energia observado para cada cliente.

**Consume_min:** Representa o mínimo de consumo de energia registado para cada cliente.

**Consume_mean_Morning, Consume_mean_Noon, Consume_mean_Evening, Consume_mean_Night:** Estas colunas mostram a média do consumo de energia para cada período do dia (manhã, tarde, noite e madrugada).

**Min_Day:** Indica o dia com o menor consumo total de energia para cada cliente.

**Max_Day:** Representa o dia com o maior consumo total de energia para cada cliente.

Estas estatísticas agregadas fornecem uma visão geral do comportamento de consumo de energia dos clientes.

### Aplicação do Modelo de Clustering
Aplicação do modelo de clustering previamente treinado aos dados processados.

. O modelo de agrupamento foi carregado e aplicado aos dados agregados `final_data_all`, gerando assim previsões para cada ponto de dados
. Estas previsões serão utilizadas para entender melhor os padrões de consumo de energia e identificar grupos de consumidores com comportamentos semelhantes.


. O modelo de agrupamento foi carregado e aplicado aos dados agregados `final_data_all`, gerando assim previsões para cada ponto de dados
. Estas previsões serão utilizadas para entender melhor os padrões de consumo de energia e identificar grupos de consumidores com comportamentos semelhantes.

In [19]:
model_path = "clustering_model"
model = PipelineModel.load(model_path)

# Step 3: Apply the model to the validation data
predictions = model.transform(final_data_all)



Criação de um DataFrame para visualizar os centros dos clusters.

In [20]:
# Extracting the KMeans model from the pipeline
kmeans_model = model.stages[-1]  # Assuming KMeans is the last stage in your pipeline

# Extract cluster centers
centers = kmeans_model.clusterCenters()
# List to hold the data
data = []

# Iterate over the cluster centers and create a Row for each center
for i, center in enumerate(centers):
    row_data = {
        "Consume_mean": float(center[0]),
        "Consume_max": float(center[1]),
        "Consume_min": float(center[2]),
        "Consume_mean_Morning": float(center[3]),
        "Consume_mean_Noon": float(center[4]),
        "Consume_mean_Evening": float(center[5]),
        "Consume_mean_Night": float(center[6]),
        "Min_Day": float(center[7]),
        "Max_Day": float(center[8])
    }
    # Append the Row to the data list
    data.append(Row(**row_data))

# Define the schema for the DataFrame
schema = StructType([
    StructField("Consume_mean", FloatType(), nullable=True),
    StructField("Consume_max", FloatType(), nullable=True),
    StructField("Consume_min", FloatType(), nullable=True),
    StructField("Consume_mean_Morning", FloatType(), nullable=True),
    StructField("Consume_mean_Noon", FloatType(), nullable=True),
    StructField("Consume_mean_Evening", FloatType(), nullable=True),
    StructField("Consume_mean_Night", FloatType(), nullable=True),
    StructField("Min_Day", FloatType(), nullable=True),
    StructField("Max_Day", FloatType(), nullable=True)
])

# Create DataFrame using the specified schema
cluster_centers_df = spark.createDataFrame(data, schema=schema)

# Show the DataFrame
cluster_centers_df.show()

+------------+-----------+-----------+--------------------+-----------------+--------------------+------------------+----------+-----------+
|Consume_mean|Consume_max|Consume_min|Consume_mean_Morning|Consume_mean_Noon|Consume_mean_Evening|Consume_mean_Night|   Min_Day|    Max_Day|
+------------+-----------+-----------+--------------------+-----------------+--------------------+------------------+----------+-----------+
|  0.05330972| 0.12908413|0.021310782|         0.057795502|       0.06503283|          0.06961891|      0.0130209215|0.04633711|0.061931286|
|   0.2937032|  0.3902783| 0.11512419|          0.32474813|        0.3505345|          0.35116425|        0.08608117|0.24990796| 0.32726064|
|  0.12606068| 0.24579746|0.050768673|          0.13470587|       0.15380688|          0.16171429|        0.03264286|0.11156876| 0.14771971|
+------------+-----------+-----------+--------------------+-----------------+--------------------+------------------+----------+-----------+



In [21]:
predictions.show(5, truncate=False,vertical=True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 LCLid                | MAC000002                                                                                                                                                                            
 Consume_mean         | 0.24523506                                                                                                                                                                           
 Consume_max          | 2.994                                                                                                                                                                                
 Consume_min          | 0.065                                                                                                                                                   

#### Centros dos Clusters
Os centros dos clusters indicam três grupos principais de consumidores com diferentes padrões de consumo médio, máximo, e mínimo, bem como variações nos períodos do dia e nos dias de menor e maior consumo.

**Cluster 1:** Consumidores com baixo consumo médio e variação mínima ao longo do dia. 

**Cluster 2:** Consumidores com consumo médio mais elevado e maior variação ao longo do dia, especialmente à noite.

**Cluster 3:** Consumidores com consumo médio moderado e variação considerável entre os diferentes períodos do dia.

### Conclusão:

Após a análise dos clusters gerados pelo modelo de agrupamento, pudemos identificar distintos perfis de consumo de energia entre os clientes. 

No Cluster 1, observam-se consumidores com baixo consumo médio e picos moderados durante o dia. Já no Cluster 2, encontramos consumidores com alto consumo médio e picos de consumo mais acentuados em determinados períodos do dia. Por fim, o Cluster 3 representou consumidores com consumo médio e padrões mais uniformes ao longo do dia.

Essa segmentação permite às empresas de energia compreender melhor o comportamento dos seus clientes e adaptar as suas estratégias de marketing e fornecimento de energia de acordo com as necessidades de cada grupo. Além disso, possibilita a identificação de oportunidades de eficiência energética e a oferta de serviços personalizados, visando tanto a satisfação do cliente quanto a maximização dos lucros.

### Resposta às questões principais:


1. **Como é que podem ser agrupados os consumidores residenciais com base nos seus comportamentos típicos de consumo de eletricidade?**
   Podemos agrupar os consumidores residenciais com base nos seus padrões de consumo de eletricidade utilizando o algoritmo K-means. Este algoritmo divide os consumidores em clusters com base na similaridade dos seus padrões de consumo, permitindo uma segmentação eficaz.

2. **Quantos clusters ou grupos distintos de consumidores podem ser identificados?**
   Utilizando o método do cotovelo e o índice de Silhouette, determinamos o número ideal de clusters que melhor representa a estrutura dos dados. No nosso caso, identificamos que o número ótimo de clusters é 3.

3. **Quais são as características típicas de cada grupo de consumidore?** Ao analisar os clusters identificados, percebemos que cada grupo de consumidores apresenta padrões de consumo únicos. No Cluster 1, observamos um perfil de consumidores com baixo consumo médio, mas com picos moderados durante o dia. No Cluster 2, encontramos consumidores com um consumo médio mais elevado, caracterizado por picos de consumo mais acentuados em determinados períodos do dia. Por fim, no Cluster 3, identificamos consumidores com consumo médio e padrões de consumo mais uniformes ao longo do dia.da grupo.

4. **Como é que os fornecedores de energia podem usar estas informações para compreender melhor os consumidores e tomar decisões mais informadas?**
   Os fornecedores de energia podem utilizar estas informações para melhor compreender as necessidades e preferências dos consumidores. Por exemplo, podem personalizar as suas estratégias de marketing e oferta de serviços com base nos diferentes perfis de consumo identificados em cada grupo. Além disso, podem desenvolver estratégias para gerir a energia de forma ma. Como por exemplo,ciente, como incentivando a adoção de medidas de eficiência energética específicas para cada grupo. Desta forma, os fornecedores de energia podem tomar decisões mais informadas e eficazes para melhor atender às necessidades dos consumidores.